9. We will now consider the `Boston` housing data set, from the `ISLR2` library.

In [2]:
library(ISLR2)
library(MASS)
library(ggplot2)
library(ggthemes)
library(boot)
library(skimr)
library(caret)

# Define a color cycle to use
colors <- colorblind_pal()(8)



Attaching package: ‘MASS’


The following object is masked from ‘package:ISLR2’:

    Boston


Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:boot’:

    melanoma




In [3]:
head(Boston)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
6,0.02985,0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


In [4]:
skim(Boston)

── Data Summary ────────────────────────
                           Values
Name                       Boston
Number of rows             506   
Number of columns          14    
_______________________          
Column type frequency:           
  numeric                  14    
________________________         
Group variables            None  

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable n_missing complete_rate     mean      sd        p0      p25
 1 crim                  0             1   3.61     8.60    0.00632   0.0820
 2 zn                    0             1  11.4     23.3     0         0     
 3 indus                 0             1  11.1      6.86    0.46      5.19  
 4 chas                  0             1   0.0692   0.254   0         0     
 5 nox                   0             1   0.555    0.116   0.385     0.449 
 6 rm                    0             1   6.28     0.703   3.56      5.89  
 7 age                   0      

In [5]:
str(Boston)

'data.frame':	506 obs. of  14 variables:
 $ crim   : num  0.00632 0.02731 0.02729 0.03237 0.06905 ...
 $ zn     : num  18 0 0 0 0 0 12.5 12.5 12.5 12.5 ...
 $ indus  : num  2.31 7.07 7.07 2.18 2.18 2.18 7.87 7.87 7.87 7.87 ...
 $ chas   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ nox    : num  0.538 0.469 0.469 0.458 0.458 0.458 0.524 0.524 0.524 0.524 ...
 $ rm     : num  6.58 6.42 7.18 7 7.15 ...
 $ age    : num  65.2 78.9 61.1 45.8 54.2 58.7 66.6 96.1 100 85.9 ...
 $ dis    : num  4.09 4.97 4.97 6.06 6.06 ...
 $ rad    : int  1 2 2 3 3 3 5 5 5 5 ...
 $ tax    : num  296 242 242 222 222 222 311 311 311 311 ...
 $ ptratio: num  15.3 17.8 17.8 18.7 18.7 18.7 15.2 15.2 15.2 15.2 ...
 $ black  : num  397 397 393 395 397 ...
 $ lstat  : num  4.98 9.14 4.03 2.94 5.33 ...
 $ medv   : num  24 21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 ...


(a) Based on this data set, provide an estimate for the population mean of `medv`. Call this estimate $\hat{\mu}$.

In [6]:
mu_hat <- mean(Boston$medv)
mu_hat

[1] 22.53281

(b) Provide an estimate of the standard error of $\hat{\mu}$. Interpret this result.
Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.

In [7]:
se <- sd(Boston$medv) / sqrt(nrow(Boston))
se

[1] 0.4088611

The standard error of the mean is a measure of the dispersion of sample means around the population mean.
In other words, how much it deviates the sample mean from the actual value.

(c) Now estimate the standard error of $\hat{\mu}$ using the bootstrap. 
How does this compare to your answer from (b)?

In [8]:
boot_mean <- function(df, index) {
    mu <- mean(df$medv[index])
    return(mu)
}

In [9]:
boot(Boston, boot_mean, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Boston, statistic = boot_mean, R = 1000)


Bootstrap Statistics :
    original      bias    std. error
t1* 22.53281 0.006826877   0.4100195

(d) Based on your bootstrap estimate from (c), provide a 95% confidence interval for the mean of `medv`. 
Compare it to the results obtained using `t.test(Boston$medv)`.
Hint: You can approximate a 95% confidence interval using the formula $[\hat{\mu} − 2\cdot\text{SE}(\hat{\mu}), \hat{\mu} + 2\cdot\text{SE}(\hat{\mu})]$.

In [11]:
confidence_interval_95 <- mu_hat + c(-1, 1) * 2 * 0.3944739
print(confidence_interval_95)

[1] 21.74386 23.32175


In [12]:
t.test(Boston$medv)


	One Sample t-test

data:  Boston$medv
t = 55.111, df = 505, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 21.72953 23.33608
sample estimates:
mean of x 
 22.53281 


Our estimate using the bootstrap method gives a slightly broader confidence interval than the $t$-test.

(e) Based on this data set, provide an estimate, $\hat{\mu}_{\text{med}}$, for the median value of `medv` in the population.

In [13]:
mu_hat_med <- median(Boston$medv)
print(mu_hat_med)

[1] 21.2


(f) We now would like to estimate the standard error of $\hat{\mu}_{\text{med}}$.
Unfortunately, there is no simple formula for computing the standard error of the median. 
Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

In [14]:
boot_median <- function(df, index) {
    mu_med <- median(df$medv[index])
    return(mu_med)
}

In [15]:
boot(Boston, boot_median, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Boston, statistic = boot_median, R = 1000)


Bootstrap Statistics :
    original  bias    std. error
t1*     21.2 -0.0325   0.3717081

The median obtained using the bootstrap method is 21.2, the same as the formula.
The standard error of the median from the bootstrap is 0.371.
Seems like a powerful method to estimate standard errors for statistics for which there are no explicit equations.

(g) Based on this data set, provide an estimate for the tenth percentile of `medv` in `Boston` census tracts. 
Call this quantity $\hat{\mu}_{0.1}$. (You can use the `quantile()` function.)

In [19]:
quantile(Boston$medv, probs = 0.1)

10% 
12.75

(h) Use the bootstrap to estimate the standard error of $\hat{\mu}_{0.1}$. Comment on your findings.

In [21]:
boot_quantile <- function(df, index) {
    mu_q <- quantile(df$medv[index], probs = 0.1)
    return(mu_q)
}

In [22]:
boot(Boston, boot_quantile, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Boston, statistic = boot_quantile, R = 1000)


Bootstrap Statistics :
    original  bias    std. error
t1*    12.75  0.0149   0.5010829